# RNN을 이용한 텍스트 생성(Text Generation using RNN)

### RNN을 이용하여 텍스트 생성하기

#### 1) 데이터에 대한 이해와 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [4]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [6]:
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기 : %d' %vocab_size)

단어 집합의 크기 : 12


In [7]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [9]:
sequences = list()
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('학습에 사용할 샘플의 개수 : %d' %len(sequences))

학습에 사용할 샘플의 개수 : 11


In [10]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [11]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [12]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [13]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [14]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [16]:
print(X) # 각 리스트 마지막 값 제외

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [15]:
print(y) # 각 리스트의 마지막 값

[ 3  1  4  5  1  7  1  9 10  1 11]


In [17]:
y = to_categorical(y, num_classes=vocab_size)

In [18]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

#### 2) 모델 설계하기

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [24]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimaizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Train on 11 samples
Epoch 1/200
11/11 - 3s - loss: 2.5058 - accuracy: 0.0000e+00
Epoch 2/200
11/11 - 0s - loss: 2.4604 - accuracy: 0.1818
Epoch 3/200
11/11 - 0s - loss: 2.4261 - accuracy: 0.2727
Epoch 4/200
11/11 - 0s - loss: 2.3930 - accuracy: 0.2727
Epoch 5/200
11/11 - 0s - loss: 2.3591 - accuracy: 0.3636
Epoch 6/200
11/11 - 0s - loss: 2.3236 - accuracy: 0.3636
Epoch 7/200
11/11 - 0s - loss: 2.2867 - accuracy: 0.3636
Epoch 8/200
11/11 - 0s - loss: 2.2485 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.2096 - accuracy: 0.3636
Epoch 10/200
11/11 - 0s - loss: 2.1709 - accuracy: 0.3636
Epoch 11/200
11/11 - 0s - loss: 2.1332 - accuracy: 0.3636
Epoch 12/200
11/11 - 0s - loss: 2.0973 - accuracy: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.0640 - accuracy: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.0336 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.0062 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 1.9819 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 1.9601 - accuracy: 0.3636

Epoch 142/200
11/11 - 0s - loss: 0.3008 - accuracy: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.2947 - accuracy: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.2886 - accuracy: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2827 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2768 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2711 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2653 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2598 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2542 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2488 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2434 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2381 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2329 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.2278 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2228 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2179 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2130 - accuracy: 1.00

In [36]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
         # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            if index == result:
                break
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    sentence = init_word + sentence
    return sentence

In [37]:
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측
print(sentence_generation(model, t, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [39]:
# 2번 예측
print(sentence_generation(model, t, '그의', 2))

그의 말이 법이다


In [48]:
print(sentence_generation(model, t, '가는', 4))

가는 말이 고와야 오는 말이


### LSTM을 이용하여 텍스트 생성하기

#### 1) 데이터에 대한 이해와 전처리

In [49]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [51]:
df=pd.read_csv('C:/Users/it/Downloads/dataset/nyt-comments/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [61]:
print('열의 개수 : ', len(df.columns))
print(df.columns)
print('행의 개수 : ', len(df))

열의 개수 :  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
행의 개수 :  1324


In [55]:
df['headline'].isnull().values.any()

False

In [57]:
headline = []
headline.extend(list(df.headline.values))
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [58]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


노이즈 데이터를 제거하기 전 데이터의 개수는 1,324입니다.  
즉, 신문 기사의 제목이 총 1,324개입니다.

In [62]:
headline = [n for n in headline if n != 'Unknown']
print('노이즤 값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즤 값 제거 후 샘플의 개수 : 1214


In [63]:
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.']

이제 데이터 전처리를 수행합니다.  
여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다.

In [68]:
def repreprocessing(s):
    s = s.encode("utf-8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

이제 단어 집합(vocabulary)을 만들고 크기를 확인합니다.

In [70]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' %vocab_size)

단어 집합의 크기 : 3494


In [72]:
sequences = list()

for line in text:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:15]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3],
 [100, 3, 1117],
 [100, 3, 1117, 2],
 [100, 3, 1117, 2, 14],
 [100, 3, 1117, 2, 14, 583]]

[[99, 269], # former nfl  
 [99, 269, 371], # former nfl cheerleaders  
 [99, 269, 371, 1115], # former nfl cheerleaders settlement  
 [99, 269, 371, 1115, 582], # former nfl cheerleaders settlement offer  
 [99, 269, 371, 1115, 582, 52], # 'former nfl cheerleaders settlement offer 1  
 [99, 269, 371, 1115, 582, 52, 7], # former nfl cheerleaders settlement offer 1 and  
 [99, 269, 371, 1115, 582, 52, 7, 2], # ... 이하 생략 ...  
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],  
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],  
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], # 모든 단어가 사용된 완전한 첫번째 문장  
 
 바로 위의 줄 : former nfl cheerleaders settlement offer 1 and a meeting with goodell  
 [100, 3]] # epa to에 해당되며 두번째 문장이 시작됨.  

In [77]:
index_to_word={}
for key, value in t.word_index.items():
    index_to_word[value] = key
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))
print('빈도수 상위 2번 단어 : {}'.format(index_to_word[2]))
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 582번 단어 : offer
빈도수 상위 2번 단어 : a
빈도수 상위 1번 단어 : the


In [79]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [80]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115  582]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582   52]]


In [82]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [83]:
print(X[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582]]


In [84]:
print(y[:5])

[ 269  371 1115  582   52]


In [85]:
y = to_categorical(y, num_classes=vocab_size)

#### 2) 모델 설계하기

In [89]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [91]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Train on 7803 samples
Epoch 1/200
7803/7803 - 9s - loss: 7.6569 - accuracy: 0.0254
Epoch 2/200
7803/7803 - 7s - loss: 7.1137 - accuracy: 0.0311
Epoch 3/200
7803/7803 - 7s - loss: 6.9731 - accuracy: 0.0340
Epoch 4/200
7803/7803 - 6s - loss: 6.8459 - accuracy: 0.0423
Epoch 5/200
7803/7803 - 7s - loss: 6.7026 - accuracy: 0.0440
Epoch 6/200
7803/7803 - 8s - loss: 6.5365 - accuracy: 0.0465
Epoch 7/200
7803/7803 - 7s - loss: 6.3510 - accuracy: 0.0487
Epoch 8/200
7803/7803 - 7s - loss: 6.1567 - accuracy: 0.0534
Epoch 9/200
7803/7803 - 7s - loss: 5.9647 - accuracy: 0.0595
Epoch 10/200
7803/7803 - 7s - loss: 5.7827 - accuracy: 0.0647
Epoch 11/200
7803/7803 - 6s - loss: 5.6093 - accuracy: 0.0713
Epoch 12/200
7803/7803 - 6s - loss: 5.4460 - accuracy: 0.0754
Epoch 13/200
7803/7803 - 6s - loss: 5.2874 - accuracy: 0.0818
Epoch 14/200
7803/7803 - 6s - loss: 5.1365 - accuracy: 0.0856
Epoch 15/200
7803/7803 - 6s - loss: 4.9942 - accuracy: 0.0969
Epoch 16/200
7803/7803 - 7s - loss: 4.8518 - accuracy: 0.

Epoch 133/200
7803/7803 - 6s - loss: 0.3544 - accuracy: 0.9153
Epoch 134/200
7803/7803 - 6s - loss: 0.3552 - accuracy: 0.9141
Epoch 135/200
7803/7803 - 6s - loss: 0.3570 - accuracy: 0.9116
Epoch 136/200
7803/7803 - 6s - loss: 0.3432 - accuracy: 0.9155
Epoch 137/200
7803/7803 - 6s - loss: 0.3405 - accuracy: 0.9145
Epoch 138/200
7803/7803 - 7s - loss: 0.3343 - accuracy: 0.9157
Epoch 139/200
7803/7803 - 6s - loss: 0.3317 - accuracy: 0.9140
Epoch 140/200
7803/7803 - 6s - loss: 0.3283 - accuracy: 0.9149
Epoch 141/200
7803/7803 - 6s - loss: 0.3252 - accuracy: 0.9154
Epoch 142/200
7803/7803 - 6s - loss: 0.3242 - accuracy: 0.9157
Epoch 143/200
7803/7803 - 6s - loss: 0.3220 - accuracy: 0.9138
Epoch 144/200
7803/7803 - 6s - loss: 0.3201 - accuracy: 0.9146
Epoch 145/200
7803/7803 - 6s - loss: 0.3168 - accuracy: 0.9166
Epoch 146/200
7803/7803 - 6s - loss: 0.3202 - accuracy: 0.9148
Epoch 147/200
7803/7803 - 7s - loss: 0.3139 - accuracy: 0.9152
Epoch 148/200
7803/7803 - 7s - loss: 0.3087 - accuracy:

In [93]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [94]:
print(sentence_generation(model, t, 'i', 10))

i cant jump ship from facebook yet help from the instagram


In [95]:
print(sentence_generation(model, t, 'how', 10))

how to make a crossword puzzle floods schools turning point no
